In [9]:
import pandas as pd
import numpy as np
import requests
import json
import os
import sqlalchemy
from sqlalchemy import create_engine
import time
from tqdm.notebook import tqdm
from io import StringIO
from google.cloud import storage

In [10]:
#setup redshift conncetion
REDSHIFT_ENDPOINT = 'juntian-cluster.cl4tv7kd0trb.us-east-1.redshift.amazonaws.com:5439/dev'
REDSHIFT_USER = 'awsuser'
REDSHIFT_PASS = 'arn:aws:secretsmanager:us-east-1:381492032483:secret:redshift!different-states-hospital-price-awsuser-FrEDZG'
REDSHIFT_PORT = '5439'
REDSHIFT_DB = 'different-states-hospital-price'

# Create the connection
connection = f"postgresql+psycopg2://{REDSHIFT_USER}:{REDSHIFT_PASS}@{REDSHIFT_ENDPOINT}:{REDSHIFT_PORT}/{REDSHIFT_DB}"
        
#Create engine
engine = create_engine(connection)

In [13]:
#load cpt_hcpcs.json into dataframe
# Set service account key
service_account_key_path = 'different-state-hospital-price-fd662d2f48c2.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_key_path

# Specify JSON file path
bucket_path = 'gs://different-state-hospital-prices/cpt_hcpcs.json'

# Read JSON file into DataFrame
df_cpt_hcpcs = pd.read_json(bucket_path, lines=True)

print(df_cpt_hcpcs.info())
df_cpt_hcpcs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287818 entries, 0 to 3287817
Data columns (total 3 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code               object
 1   short_description  object
 2   long_description   object
dtypes: object(3)
memory usage: 75.3+ MB
None


,code,short_description,long_description
0,00000A,DVC REVASC 6X20MM 200CM,None
1,00001U,RBC DNA HEA 35 AG PLA,None
2,"00001U,1",RBC DNA HEA 35 AG PLA,None
3,00013,PT INDIVIDUAL GYM,None
4,0001A,HC ADM PFIZER SARSCOV2 30MCG/0.3ML 1ST,None


In [14]:
#cpt_hcpcs transformation
df_cpt_hcpcs = df_cpt_hcpcs.astype({
    'code': 'string',
    'short_description': 'string',
    'long_description': 'string'
})
df_cpt_hcpcs = df_cpt_hcpcs.fillna('None')

df_cpt_hcpcs = df_cpt_hcpcs.drop_duplicates()

print(df_cpt_hcpcs.info())
df_cpt_hcpcs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287818 entries, 0 to 3287817
Data columns (total 3 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code               string
 1   short_description  string
 2   long_description   string
dtypes: string(3)
memory usage: 75.3 MB
None


,code,short_description,long_description
0,00000A,DVC REVASC 6X20MM 200CM,None
1,00001U,RBC DNA HEA 35 AG PLA,None
2,"00001U,1",RBC DNA HEA 35 AG PLA,None
3,00013,PT INDIVIDUAL GYM,None
4,0001A,HC ADM PFIZER SARSCOV2 30MCG/0.3ML 1ST,None


In [19]:
#load hospitals.json into dataframe
bucket_path = 'gs://different-state-hospital-prices/hospitals.json'
df_hospitals = pd.read_json(bucket_path, lines=True)

print(df_hospitals.info())
df_hospitals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   npi_number      1400 non-null   object 
 1   name            1400 non-null   object 
 2   url             1400 non-null   object 
 3   street_address  1351 non-null   object 
 4   city            1379 non-null   object 
 5   state           1377 non-null   object 
 6   zip_code        1357 non-null   object 
 7   publish_date    610 non-null    float64
dtypes: float64(1), object(7)
memory usage: 87.6+ KB
None


,npi_number,name,url,street_address,city,state,zip_code,publish_date
0,1003139775.0,HCA Virginia,https://hcavirginia.com/about/legal/pricing-tr...,901 E. Cary St Suite 210,Richmond,VA,None,1.609459e+12
1,1003260480,Brookwood Baptist Medical Center,https://www.brookwoodbaptisthealth.com/docs/gl...,2010 Brookwood Medical Center Dr.,Birmingham,AL,35209,NaN
2,1003281452,Henderson Hospital,https://uhsfilecdn.eskycity.net/ac/henderson-h...,1050 West Galleria Drive,Henderson,NV,89011,1.609459e+12
3,1003362997,CHI Health St. Elizabeth,https://www.chihealth.com/content/dam/chi-heal...,555 S. 70Th St.,Lincoln,NE,68510,1.609459e+12
4,1003389206,Merrill pioneer hospital,https://www.avera.org/app/files/public/79147/m...,"1100 S 10th Ave, Ste 100",Rock Rapids,IA,51246-2020,NaN


In [22]:
#hospitals transformation
#drop the uncessary column
df_hospitals = df_hospitals.drop(columns = ['publish_date'])

#Unify npi_number
df_hospitals['npi_number'] = df_hospitals['npi_number'].astype(str)
df_hospitals['npi_number'] = df_hospitals['npi_number'].str.replace(r'\D', '', regex=True)
df_hospitals['npi_number'] = df_hospitals['npi_number'].apply(lambda x: x[:10] if len(x) >= 10 else None).astype('string')

#unify zip_code
df_hospitals['zip_code'] = df_hospitals['zip_code'].astype(str)
df_hospitals['zip_code'] = df_hospitals['zip_code'].str.replace('.0','')

# covert other types using astype
df_hospitals = df_hospitals.astype({
    'name': 'string',
    'url': 'string',
    'street_address':'string',
    'city':'string',
    'state':'string',
})

df_hospitals = df_hospitals.fillna('None')

df_hospitals = df_hospitals.drop_duplicates()
print(df_hospitals.info())
df_hospitals.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1378 entries, 0 to 1399
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   npi_number      1378 non-null   string
 1   name            1378 non-null   string
 2   url             1378 non-null   string
 3   street_address  1378 non-null   string
 4   city            1378 non-null   string
 5   state           1378 non-null   string
 6   zip_code        1378 non-null   object
dtypes: object(1), string(6)
memory usage: 86.1+ KB
None


,npi_number,name,url,street_address,city,state,zip_code
0,1003139775,HCA Virginia,https://hcavirginia.com/about/legal/pricing-tr...,901 E. Cary St Suite 210,Richmond,VA,None
1,1003260480,Brookwood Baptist Medical Center,https://www.brookwoodbaptisthealth.com/docs/gl...,2010 Brookwood Medical Center Dr.,Birmingham,AL,35209
2,1003281452,Henderson Hospital,https://uhsfilecdn.eskycity.net/ac/henderson-h...,1050 West Galleria Drive,Henderson,NV,89011
3,1003362997,CHI Health St. Elizabeth,https://www.chihealth.com/content/dam/chi-heal...,555 S. 70Th St.,Lincoln,NE,68510
4,1003389206,Merrill pioneer hospital,https://www.avera.org/app/files/public/79147/m...,"1100 S 10th Ave, Ste 100",Rock Rapids,IA,51246-2020


In [23]:
#load prices.json into dataframe
bucket_path = 'gs://different-state-hospital-prices/prices.json'

df_prices = pd.DataFrame()

# Create a JSON reader
json_reader = pd.read_json(bucket_path, lines=True, chunksize=10000)

# Process each chunk
for chunk in json_reader:
    # Clean 'npi_number' column
    chunk['npi_number'] = chunk['npi_number'].astype(str)
    chunk['npi_number'] = chunk['npi_number'].str.replace(r'\D', '', regex=True)
    chunk['npi_number'] = chunk['npi_number'].apply(lambda x: x[:10] if len(x) >= 10 else None).astype('string')

    # change dtype
    chunk = chunk.astype({
        'code': 'string',
        'payer': 'string'
    })
    
    # Append to dataframe
    df_prices = pd.concat([df_prices, chunk], ignore_index=True)
    
    print(len(df_prices))
    break
    
df_prices = df_prices.fillna('none')

df_prices = df_prices.drop_duplicates()

print(df_prices.head())
print(df_prices.info())

10000
     code  npi_number payer     price
0  00000A  1053358010  CASH  75047.00
1  00000A  1336186394  CASH  75047.00
2  00001U  1003139775  CASH    457.23
3  00001U  1053824292  CASH    972.00
4  00001U  1417901406  CASH    296.00
<class 'pandas.core.frame.DataFrame'>
Index: 9971 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   code        9971 non-null   string 
 1   npi_number  9971 non-null   string 
 2   payer       9971 non-null   string 
 3   price       9971 non-null   float64
dtypes: float64(1), string(3)
memory usage: 389.5 KB
None


In [28]:
#parper for load
# Adding surrogate keys for each dimension table
df_cpt_hcpcs['cpt_id'] = np.arange(1, len(df_cpt_hcpcs) + 1)
df_hospitals['hospital_id'] = np.arange(1, len(df_hospitals) + 1)

# Mapping foreign keys in the fact table
df_prices['cpt_id'] = df_prices['code'].map(df_cpt_hcpcs.set_index('code')['cpt_id'])
df_prices['hospital_id'] = df_prices['npi_number'].map(df_hospitals.set_index('npi_number')['hospital_id'])

# Creating a unique list of payers and mapping payer_id
unique_payers = pd.DataFrame(df_prices['payer'].unique(), columns=['payer'])
unique_payers['payer_id'] = np.arange(1, len(unique_payers) + 1)
df_prices['payer_id'] = df_prices['payer'].map(unique_payers.set_index('payer')['payer_id'])

# Prepare final DataFrames for the new schema
dim_cpt_hcpcs = df_cpt_hcpcs[['cpt_id', 'code', 'short_description', 'long_description']]
dim_hospitals = df_hospitals[['hospital_id', 'npi_number', 'name', 'url', 'street_address', 'city', 'state', 'zip_code']]
dim_payer = unique_payers[['payer_id', 'payer']]
fact_prices = df_prices[['cpt_id', 'hospital_id', 'payer_id', 'price']]

print(dim_cpt_hcpcs.info())
print(dim_hospitals.info())
print(dim_payer.info())
print(fact_prices.info())

InvalidIndexError: Reindexing only valid with uniquely valued Index objects